In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install laserembeddings

     |████████████████████████████████| 860kB 6.8MB/s 
     |████████████████████████████████| 51kB 6.2MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=6dc99a3f01efeba31d495f4a3170d8afad7f2fcd7f5406ce1ff51da812449eef
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [ ]:
!python -m laserembeddings download-models


✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fcodes    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/93langs.fvocab    
✅   Downloaded https://dl.fbaipublicfiles.com/laser/models/bilstm.93langs.2018-12-26.pt    

✨ You're all set!


In [ ]:
!pip install emoji

     |████████████████████████████████| 51kB 2.7MB/s 
  Created wheel for emoji: filename=emoji-0.6.0-cp36-none-any.whl size=49716 sha256=4f37c3b1a1856becfccbf16b4f1e17641cc99e4fd8d47995df3f6ab6b23b7dd1
  Stored in directory: /root/.cache/pip/wheels/46/2c/8b/9dcf5216ca68e14e0320e283692dce8ae321cdc01e73e17796
Successfully built emoji


In [ ]:
import pandas as pd
import numpy as np
import os
from pandas import DataFrame
from laserembeddings import Laser
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
import pickle

from statistics import mode

import re
import emoji
import random

import nltk
nltk.download('stopwords')
from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, TweetTokenizer
from nltk.corpus import wordnet, stopwords

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
def preprocess(df):
  
  #removes URL
  pattern = r'https.?://[^\s]+[\s]?'
  df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
  
  #removes usernames/mentions
  pattern = r'@[^\s]+'
  df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
  
  #removes emoji and smiley
  pattern = re.compile("["
                      u"\U0001F600-\U0001F64F"
                      u"\U0001F300-\U0001F5FF"
                      u"\U0001F680-\U0001F6FF"
                      u"\U0001F1E0-\U0001F1FF"
                      u"\U00002500-\U00002BEF"
                      u"\U00002702-\U000027B0"
                      u"\U00002702-\U000027B0"
                      u"\U000024C2-\U0001F251"
                      u"\U0001f926-\U0001f937"
                      u"\U00010000-\U0010ffff"
                      u"\u2640-\u2642"
                      u"\u2600-\u2B55"
                      u"\u200d"
                      u"\u23cf"
                      u"\u23e9"
                      u"\u231a"
                      u"\ufe0f"
                      u"\u3030"
                      "]+", flags=re.UNICODE)
  df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
  
  #removes numbers
  pattern = r'\d+'
  df["tweet"] = df["tweet"].str.replace(pat=pattern, repl="", regex=True)
  
  #removes punctuation
  pattern = r"[^\w\s]"
  df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)

  #removes stop words
  stop_words = stopwords.words("english")    
  remove_stop_words = lambda row: " ".join([token for token in row.split(" ")
                                            if token not in stop_words])
  df["tweet"] = df["tweet"].apply(remove_stop_words)
  
  #removes extra spaces
  pattern = r"[\s]+"
  df["tweet"] = df["tweet"].str.replace(pat=pattern, repl=" ", regex=True)
  
  return(df)

In [ ]:
lang = ['Arabic','English','French']
cols = []
for i in range(1024):
  cols.append(i)

In [ ]:
def preprocessing(language):
    path = 'drive/My Drive/CS695'
    data = pd.read_csv(os.path.join(path,language+'.csv'))
    data_preprocess = preprocess(data)
    data_to_list = data_preprocess['tweet'].to_list()

    laser = Laser()
    embeddings = laser.embed_sentences(data_to_list, lang='en') 

    df1=pd.DataFrame(embeddings)
    df2 = pd.DataFrame(data['label'])

    sss1 = StratifiedShuffleSplit(n_splits=1,test_size=0.3,random_state=0)
    a = list(sss1.split(df1, df2))
    train_x,interX = df1.iloc[a[0][0],:],df1.iloc[a[0][1],:]
    train_y,interY = df2.iloc[a[0][0],:],df2.iloc[a[0][1],:]

    sss2 = StratifiedShuffleSplit(n_splits=1,test_size=0.35,random_state=0)
    a = list(sss2.split(interX, interY))
    test_x,val_x = interX.iloc[a[0][0],:],interX.iloc[a[0][1],:]
    test_y,val_y = interY.iloc[a[0][0],:],interY.iloc[a[0][1],:]

    return train_x, test_x, val_x, train_y, test_y, val_y 

In [ ]:
import io
from io import StringIO 
test_string = """tweet,label
                 !!!!Click below for a FREE download of a colorfully illustrated 132 page e-book on the Zionist-engineered INTENTIONAL destruction of Western civilization, 1
                 In my opinion using cuss words like bitch is demeaning to women, 0
                 @usernew!!!! I hope they all starve, 1"""

data = io.StringIO(test_string)
df = pd.read_csv(data, sep=",")
df = preprocess(df)
print(df)

                                               tweet  label
0   Click FREE download colorfully illustrated pa...      1
1   In opinion using cuss words like bitch demean...      0
2                                      I hope starve      1


In [ ]:
data_to_list = df['tweet'].to_list()

laser = Laser()
embeddings = laser.embed_sentences(data_to_list, lang='en') 

new_test_x =pd.DataFrame(embeddings)
new_test_y = pd.DataFrame(df['label'])

In [ ]:
Arabic_train_x, Arabic_test_x, Arabic_val_x, Arabic_train_y, Arabic_test_y, Arabic_val_y = preprocessing('Arabic')
English_train_x, English_test_x, English_val_x, English_train_y, English_test_y, English_val_y = preprocessing('English')
French_train_x, French_test_x, French_val_x, French_train_y, French_test_y, French_val_y = preprocessing('French')

In [ ]:
_English = pd.concat([English_train_x, English_train_y], axis=1)
_Arabic = pd.concat([Arabic_train_x, Arabic_train_y], axis=1)
_French = pd.concat([French_train_x, French_train_y], axis=1)

In [ ]:
df_ = pd.concat([_English, _Arabic], axis=0)
df_ = pd.concat([df_, _French], axis= 0)

In [ ]:
def model_for_all(c, seed):
  
  df_new = df_.sample(frac=1,random_state=seed).reset_index(drop=True)
            
  df_x = df_new[cols]
  df_y = df_new['label']

  model = LogisticRegression(C=c,solver='lbfgs',class_weight='balanced',random_state=seed)
  model.fit(df_x, df_y)
    
  error_analysis = model.predict(new_test_x)
  print(error_analysis)
 

In [ ]:
model_for_all(0.1,2018)

[0 1 0]
